### File Selection
This is where you will simply specifify the RDFS and instance model you would like to validate.

### RDFS Parsing
This cell parses the RDFS file and builds a Python structure describing classes, associations, and multiplicities.

In [ ]:
import tkinter as tk
from tkinter import filedialog
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output

instance_path = None
rdfs_paths = []

instance_label = widgets.Label(value="No CIM instance file selected")
rdfs_list_widget = widgets.Textarea(value="No RDFS profiles selected", layout=widgets.Layout(width='600px', height='100px'), disabled=True)
status = widgets.HTML(value="<b>Status:</b> Ready")
validate_btn = widgets.Button(description="Parse & Validate", button_style="success", disabled=True)
output = widgets.Output(layout=widgets.Layout(height='600px', overflow_y='auto', border='1px solid #ccc'))

def update_validate_button():
    if instance_path and len(rdfs_paths) > 0:
        validate_btn.disabled = False
        status.value = "<b>Status:</b> Ready to validate"
    else:
        validate_btn.disabled = True

def update_rdfs_display():
    if len(rdfs_paths) == 0:
        rdfs_list_widget.value = "No RDFS profiles selected"
    else:
        rdfs_list_widget.value = "\n".join([f"{i+1}. {p.name}" for i, p in enumerate(rdfs_paths)])

def select_instance_file(b):
    global instance_path
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    file_path = filedialog.askopenfilename(
        title="Select CIM Instance File",
        filetypes=(("XML files", "*.xml"), ("All files", "*.*"))
    )
    root.destroy()
    if file_path:
        instance_path = Path(file_path)
        instance_label.value = f"Selected: {instance_path.name}"
        update_validate_button()

def add_rdfs_file(b):
    global rdfs_paths
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    file_paths = filedialog.askopenfilenames(
        title="Select RDFS Profile File(s) - You can select multiple",
        filetypes=(("RDFS files", "*.rdfs"), ("XML files", "*.xml"), ("All files", "*.*"))
    )
    root.destroy()
    if file_paths:
        for fp in file_paths:
            p = Path(fp)
            if p not in rdfs_paths:
                rdfs_paths.append(p)
        update_rdfs_display()
        update_validate_button()

def clear_rdfs_files(b):
    global rdfs_paths
    rdfs_paths = []
    update_rdfs_display()
    update_validate_button()

def run_validation(b):
    with output:
        clear_output(wait=True)
        status.value = "<b>Status:</b> Parsing and validating..."
        
        try:
            print("="*60)
            print("STEP 1: Parsing RDFS Profiles")
            print("="*60)
            print(f"Loading {len(rdfs_paths)} profile(s):")
            for i, rp in enumerate(rdfs_paths):
                print(f"  {i+1}. {rp.name}")
            
            import xml.etree.ElementTree as ET
            
            rdfs_classes = {}
            cim_datatypes = {}
            enumerations = {}
            forward_to_inverse = {}
            class_hierarchy = {}
            expected_cim_namespace = None  # Will be extracted from RDFS xml:base
            
            ns = {
                "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
                "cims": "http://iec.ch/TC57/1999/rdf-schema-extensions-19990926#"
            }
            
            def extract_class_name(uri):
                if not uri:
                    return None
                if '#' in uri:
                    return uri.split('#')[-1]
                if '/' in uri:
                    return uri.split('/')[-1]
                return uri
            
            def parse_multiplicity(uri):
                if uri and uri.startswith("http") and "#M:" in uri:
                    val = uri.split("#M:")[-1]
                    min_, max_ = val.split("..")
                    min_ = int(min_)
                    max_ = float('inf') if max_ == 'n' else int(max_)
                    return (min_, max_)
                return (None, None)
            
            def resolve_datatype(datatype_name, cim_datatypes):
                """Resolve a CIMDatatype to its primitive type. If it's a CIMDatatype, assume Float."""
                if not datatype_name:
                    return datatype_name
                
                if datatype_name in cim_datatypes:
                    value_datatype = cim_datatypes[datatype_name].get('value_datatype')
                    if value_datatype:
                        return extract_class_name(value_datatype)
                    return 'Float'
                
                return datatype_name
            
            def validate_datatype(value, datatype_uri, cim_datatypes):
                if not datatype_uri or value is None:
                    return True, None
                
                datatype = extract_class_name(datatype_uri)
                resolved = resolve_datatype(datatype, cim_datatypes)
                
                try:
                    if resolved in ['string', 'String', 'normalizedString', 'token']:
                        return True, None
                    elif resolved in ['integer', 'int', 'Integer']:
                        int(value)
                        return True, None
                    elif resolved in ['float', 'Float', 'double', 'Double', 'decimal', 'Decimal']:
                        float(value)
                        return True, None
                    elif resolved in ['boolean', 'Boolean']:
                        if str(value).lower() not in ['true', 'false', '0', '1']:
                            return False, f"expected boolean (true/false), got '{value}'"
                        return True, None
                    elif resolved in ['dateTime', 'DateTime', 'date', 'time', 'Date']:
                        if not any(c.isdigit() for c in str(value)):
                            return False, f"expected dateTime format, got '{value}'"
                        return True, None
                    else:
                        return True, None
                except (ValueError, AttributeError, TypeError) as e:
                    return False, f"expected {resolved}, got '{value}'"
            
            def get_all_attributes(class_name, rdfs_classes, class_hierarchy):
                """Get all attributes for a class, including inherited ones."""
                attrs = {}
                current = class_name
                visited = set()
                
                while current and current not in visited:
                    visited.add(current)
                    if current in rdfs_classes:
                        attrs.update(rdfs_classes[current].get("attributes", {}))
                    current = class_hierarchy.get(current)
                
                return attrs
            
            for rdfs_path in rdfs_paths:
                print(f"\nParsing {rdfs_path.name}...")
                rdfs_tree = ET.parse(rdfs_path)
                rdfs_root = rdfs_tree.getroot()
                
                # Extract xml:base as the expected CIM namespace
                rdfs_base_uri = rdfs_root.attrib.get('{http://www.w3.org/XML/1998/namespace}base', None)
                if rdfs_base_uri and expected_cim_namespace is None:
                    expected_cim_namespace = rdfs_base_uri + '#'
                    print(f"  ✓ Detected RDFS base URI: {rdfs_base_uri}")
                    print(f"  ✓ Expected CIM namespace: {expected_cim_namespace}")
                
                profile_classes = 0
                profile_assocs = 0
                profile_attrs = 0
                profile_datatypes = 0
                profile_enum_values = 0
                
                # Pass 1: Identify CIMDatatypes
                for desc in rdfs_root.findall(".//rdf:Description", ns):
                    about = desc.attrib.get(f"{{{ns['rdf']}}}about", "")
                    stereotype_elem = desc.find("cims:stereotype", ns)
                    
                    if stereotype_elem is not None and stereotype_elem.text == "CIMDatatype":
                        datatype_name = extract_class_name(about)
                        if datatype_name:
                            cim_datatypes[datatype_name] = {}
                            profile_datatypes += 1
                
                # Pass 2: Find .value properties
                for desc in rdfs_root.findall(".//rdf:Description", ns):
                    about = desc.attrib.get(f"{{{ns['rdf']}}}about", "")
                    
                    if '.value' in about:
                        domain = desc.find("rdfs:domain", ns)
                        datatype_elem = desc.find("cims:dataType", ns)
                        
                        if domain is not None and datatype_elem is not None:
                            domain_uri = domain.attrib.get(f"{{{ns['rdf']}}}resource", "")
                            domain_name = extract_class_name(domain_uri)
                            value_datatype = datatype_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                            
                            if domain_name in cim_datatypes:
                                cim_datatypes[domain_name]['value_datatype'] = value_datatype
                
                # Pass 3: Classes, enumerations, and their parent relationships
                for desc in rdfs_root.findall(".//rdf:Description", ns):
                    about = desc.attrib.get(f"{{{ns['rdf']}}}about", "")
                    
                    if desc.find("rdf:type", ns) is not None and \
                       desc.find("rdf:type", ns).attrib.get(f"{{{ns['rdf']}}}resource", "").endswith("Class"):
                        stereotype_elem = desc.find("cims:stereotype", ns)
                        
                        if stereotype_elem is not None and stereotype_elem.text == "CIMDatatype":
                            continue
                        
                        class_name = extract_class_name(about)
                        if class_name:
                            # Check if this is an enumeration
                            if stereotype_elem is not None:
                                stereotype_resource = stereotype_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                                stereotype_text = stereotype_elem.text
                                if "enumeration" in (stereotype_resource or "") or stereotype_text == "enumeration":
                                    enumerations[class_name] = set()
                                    profile_classes += 1
                                    continue
                            
                            rdfs_classes.setdefault(class_name, {"associations": {}, "attributes": {}})
                            profile_classes += 1
                            
                            # Track parent class
                            subclass_of = desc.find("rdfs:subClassOf", ns)
                            if subclass_of is not None:
                                parent_uri = subclass_of.attrib.get(f"{{{ns['rdf']}}}resource", "")
                                parent_class = extract_class_name(parent_uri)
                                if parent_class:
                                    class_hierarchy[class_name] = parent_class
                
                # Pass 3.5: Collect enumeration values
                for desc in rdfs_root.findall(".//rdf:Description", ns):
                    about = desc.attrib.get(f"{{{ns['rdf']}}}about", "")
                    stereotype_elem = desc.find("cims:stereotype", ns)
                    label_elem = desc.find("rdfs:label", ns)
                    type_elem = desc.find("rdf:type", ns)
                    
                    # Check if this is an enum value (has stereotype="enum")
                    if stereotype_elem is not None and stereotype_elem.text == "enum":
                        enum_class = None
                        
                        # Method 1: Extract from label if it has format "EnumClass.value"
                        if label_elem is not None:
                            label_text = label_elem.text
                            if '.' in label_text:
                                enum_class, enum_value = label_text.rsplit('.', 1)
                        
                        # Method 2: Extract from rdf:type resource (for labels without dots)
                        if enum_class is None and type_elem is not None:
                            type_resource = type_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                            enum_class = extract_class_name(type_resource)
                        
                        # Add the enumeration value if we found the class
                        if enum_class and enum_class in enumerations:
                            enumerations[enum_class].add(about)  # Store full URI
                            profile_enum_values += 1
                
                # Pass 4: Properties
                for desc in rdfs_root.findall(".//rdf:Description", ns):
                    about = desc.attrib.get(f"{{{ns['rdf']}}}about", "")
                    
                    if desc.find("rdf:type", ns) is not None and \
                         desc.find("rdf:type", ns).attrib.get(f"{{{ns['rdf']}}}resource", "").endswith("Property"):
                        domain = desc.find("rdfs:domain", ns)
                        label = desc.find("rdfs:label", ns)
                        range_elem = desc.find("rdfs:range", ns)
                        datatype_elem = desc.find("cims:dataType", ns)
                        mult_elem = desc.find("cims:multiplicity", ns)
                        assoc_used_elem = desc.find("cims:AssociationUsed", ns)
                        inverse_elem = desc.find("cims:inverseRoleName", ns)
                        
                        if domain is not None and label is not None and mult_elem is not None:
                            domain_uri = domain.attrib.get(f"{{{ns['rdf']}}}resource", "")
                            domain_class = extract_class_name(domain_uri)
                            
                            if domain_class in cim_datatypes:
                                continue
                            
                            prop_name = label.text
                            multiplicity_uri = mult_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                            min_, max_ = parse_multiplicity(multiplicity_uri)
                            assoc_used = assoc_used_elem.text if assoc_used_elem is not None else "Yes"
                            
                            if domain_class and prop_name:
                                rdfs_classes.setdefault(domain_class, {"associations": {}, "attributes": {}})
                                
                                if datatype_elem is not None:
                                    datatype_uri = datatype_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                                    rdfs_classes[domain_class]["attributes"][prop_name] = {
                                        "multiplicity": (min_, max_),
                                        "datatype": datatype_uri
                                    }
                                    profile_attrs += 1
                                elif range_elem is not None:
                                    range_uri = range_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                                    range_class = extract_class_name(range_uri)
                                    
                                    if 'XMLSchema' in range_uri or '/2001/XMLSchema' in range_uri:
                                        rdfs_classes[domain_class]["attributes"][prop_name] = {
                                            "multiplicity": (min_, max_),
                                            "datatype": range_uri
                                        }
                                        profile_attrs += 1
                                    elif range_class and assoc_used == "Yes":
                                        rdfs_classes[domain_class]["associations"][prop_name] = {
                                            "multiplicity": (min_, max_),
                                            "range": range_class
                                        }
                                        profile_assocs += 1
                                        
                                        if inverse_elem is not None:
                                            inverse_uri = inverse_elem.attrib.get(f"{{{ns['rdf']}}}resource", "")
                                            inverse_full = extract_class_name(inverse_uri) if not '.' in inverse_uri else inverse_uri.lstrip("#")
                                            if '#' in inverse_uri and '.' in inverse_uri.split('#')[-1]:
                                                inverse_full = inverse_uri.split('#')[-1]
                                            
                                            if '.' in inverse_full:
                                                inv_class, inv_prop = inverse_full.split('.', 1)
                                                forward_to_inverse[f"{domain_class}.{prop_name}"] = {
                                                    "inverse_class": inv_class,
                                                    "inverse_prop": inv_prop,
                                                    "range": range_class
                                                }
                
                print(f"  ✓ {profile_classes} classes, {profile_assocs} associations, {profile_attrs} attributes, {profile_datatypes} CIM datatypes, {profile_enum_values} enum values")
            
            print(f"\n✓ Merged schema contains:")
            total_assocs = sum(len(c["associations"]) for c in rdfs_classes.values())
            total_attrs = sum(len(c["attributes"]) for c in rdfs_classes.values())
            total_enum_values = sum(len(vals) for vals in enumerations.values())
            print(f"  - {len(rdfs_classes)} total classes")
            print(f"  - {total_assocs} associations")
            print(f"  - {total_attrs} attributes")
            print(f"  - {len(cim_datatypes)} CIM datatypes")
            print(f"  - {len(class_hierarchy)} inheritance relationships")
            print(f"  - {len(forward_to_inverse)} forward-to-inverse mappings")
            print(f"  - {len(enumerations)} enumeration classes with {total_enum_values} valid values")

            print("\n" + "="*60)
            print("STEP 2: Parsing CIM Instance Data")
            print("="*60)

            # Use lxml for line number tracking
            try:
                from lxml import etree as lxml_ET
                use_lxml = True
            except ImportError:
                print("⚠️ lxml not available - line numbers will not be shown")
                use_lxml = False

            if use_lxml:
                tree = lxml_ET.parse(str(instance_path))
                root = tree.getroot()
            else:
                tree = ET.parse(instance_path)
                root = tree.getroot()

            # Extract actual namespaces from the root element
            ns_instance = {}
            if use_lxml:
                for prefix, uri in root.nsmap.items():
                    if prefix is not None:  # Skip default namespace (None)
                        ns_instance[prefix] = uri
            else:
                # ElementTree stores namespaces differently
                for key, value in root.attrib.items():
                    if key.startswith('{http://www.w3.org/2000/xmlns/}'):
                        prefix_name = key.split('}')[1]
                        ns_instance[prefix_name] = value

            # Add standard RDF namespace if not present
            if 'rdf' not in ns_instance:
                ns_instance['rdf'] = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"

            print(f"✓ Detected namespaces in instance file:")
            for prefix, uri in ns_instance.items():
                print(f"  - {prefix}: {uri}")

            # Check if CIM namespace matches RDFS base URI
            if expected_cim_namespace and 'cim' in ns_instance:
                if ns_instance['cim'] != expected_cim_namespace:
                    print(f"\n⚠️ Namespace mismatch detected:")
                    print(f"  - RDFS expects: {expected_cim_namespace}")
                    print(f"  - Instance uses: {ns_instance['cim']}")
                    print(f"  (Validation will continue, but this may cause issues)")

            instances = {}
            uri_to_line = {}

            # Parse instances with rdf:about
            for elem in root.findall(".//*[@rdf:about]", {'rdf': ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}):
                uri = elem.get(f"{{{ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}}}about")
                class_name = elem.tag.split('}')[-1]
                inst = {"class": class_name, "associations": {}, "attributes": {}}
                
                # Store line number (only works with lxml)
                if use_lxml:
                    line_num = elem.sourceline
                    if line_num:
                        uri_to_line[uri] = line_num
                
                for child in elem:
                    tag = child.tag.split('}')[-1]
                    if '.' in tag:
                        _, prop = tag.split('.', 1)
                        res = child.get(f"{{{ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}}}resource")
                        
                        if res:
                            inst["associations"].setdefault(prop, []).append(res)
                        else:
                            inst["attributes"].setdefault(prop, []).append(child.text)
                
                instances[uri] = inst

            # Parse instances with rdf:ID (convert to fragment URI format)
            for elem in root.findall(".//*[@rdf:ID]", {'rdf': ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}):
                id_value = elem.get(f"{{{ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}}}ID")
                # Create fragment URI: #ID_value
                uri = f"#{id_value}"
                class_name = elem.tag.split('}')[-1]
                inst = {"class": class_name, "associations": {}, "attributes": {}}
                
                # Store line number (only works with lxml)
                if use_lxml:
                    line_num = elem.sourceline
                    if line_num:
                        uri_to_line[uri] = line_num
                
                for child in elem:
                    tag = child.tag.split('}')[-1]
                    if '.' in tag:
                        _, prop = tag.split('.', 1)
                        res = child.get(f"{{{ns_instance.get('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')}}}resource")
                        
                        if res:
                            inst["associations"].setdefault(prop, []).append(res)
                        else:
                            inst["attributes"].setdefault(prop, []).append(child.text)
                
                instances[uri] = inst

            print(f"✓ Parsed {len(instances)} CIM instances")
            if use_lxml and uri_to_line:
                print(f"  ✓ Tracked line numbers for {len(uri_to_line)} instances")
            
            from collections import Counter
            class_counts = Counter(inst["class"] for inst in instances.values())
            print(f"  Top 5 classes: {dict(class_counts.most_common(5))}")
            
            print("\nBuilding inverse associations...")
            inverse_built_count = 0
            for uri, inst in instances.items():
                cls = inst["class"]
                for assoc_name, targets in inst["associations"].items():
                    forward_key = f"{cls}.{assoc_name}"
                    if forward_key in forward_to_inverse:
                        inv_info = forward_to_inverse[forward_key]
                        inv_prop = inv_info["inverse_prop"]
                        
                        for target_uri in targets:
                            if target_uri in instances:
                                instances[target_uri]["associations"].setdefault(inv_prop, []).append(uri)
                                inverse_built_count += 1
            
            print(f"✓ Built {inverse_built_count} inverse association links")
            
            print("\n" + "="*60)
            print("STEP 3: Validating Instance Data Against RDFS Profile")
            print("="*60)
            
            errors = []
            warnings = []
            datatype_checks = 0
            enum_checks = 0
            
            for uri, inst in instances.items():
                cls = inst["class"]
                line_info = f" (line {uri_to_line[uri]})" if uri in uri_to_line else ""
                
                # Classes not in RDFS are WARNINGS (they won't be consumed by typical consumers)
                if cls not in rdfs_classes:
                    if cls not in enumerations:
                        warnings.append(f"Class '{cls}' not defined in RDFS profile (instance: {uri}){line_info}")
                    continue
                
                # Get all attributes including inherited ones
                all_attrs = get_all_attributes(cls, rdfs_classes, class_hierarchy)
                
                # Validate associations
                assoc_defs = rdfs_classes[cls].get("associations", {})
                for assoc, defn in assoc_defs.items():
                    min_, max_ = defn["multiplicity"]
                    targets = inst["associations"].get(assoc, [])
                    actual = len(targets)
                    
                    if min_ is not None and actual < min_:
                        errors.append(f"{cls} '{uri}'{line_info} association '{assoc}' has {actual}, below minimum {min_}")
                    if max_ is not None and actual > max_:
                        errors.append(f"{cls} '{uri}'{line_info} association '{assoc}' has {actual}, above maximum {max_}")
                    
                    # Validate enumeration values in associations
                    range_class = defn.get("range")
                    if range_class and range_class in enumerations:
                        valid_values = enumerations[range_class]
                        # Extract just the enum class.value parts from valid URIs
                        valid_value_names = set()
                        for v in valid_values:
                            # Extract "EnumClass.value" from full URI
                            if '#' in v:
                                valid_value_names.add(v.split('#')[-1])
                            else:
                                valid_value_names.add(v)
                        
                        for target in targets:
                            enum_checks += 1
                            # Extract "EnumClass.value" from instance URI
                            if '#' in target:
                                target_value_name = target.split('#')[-1]
                            else:
                                target_value_name = target
                            
                            # Compare just the class.value part
                            if target_value_name not in valid_value_names:
                                valid_names_list = sorted(list(valid_value_names))
                                errors.append(f"{cls} '{uri}'{line_info} association '{assoc}' has invalid enumeration value '{target_value_name}'. Valid values for {range_class}: {', '.join(valid_names_list[:10])}{'...' if len(valid_names_list) > 10 else ''}")
                
                # Validate attributes (including inherited)
                for attr, defn in all_attrs.items():
                    min_, max_ = defn["multiplicity"]
                    datatype = defn.get("datatype")
                    actual_values = inst["attributes"].get(attr, [])
                    actual = len(actual_values)
                    
                    if min_ is not None and actual < min_:
                        errors.append(f"{cls} '{uri}'{line_info} attribute '{attr}' has {actual}, below minimum {min_}")
                    if max_ is not None and actual > max_:
                        errors.append(f"{cls} '{uri}'{line_info} attribute '{attr}' has {actual}, above maximum {max_}")
                    
                    if datatype and actual_values:
                        for val in actual_values:
                            datatype_checks += 1
                            is_valid, error_msg = validate_datatype(val, datatype, cim_datatypes)
                            if not is_valid:
                                errors.append(f"{cls} '{uri}'{line_info} attribute '{attr}': {error_msg}")
            
            print(f"Performed {datatype_checks} datatype validations and {enum_checks} enumeration validations")
            
            print("\n" + "="*60)
            print("VALIDATION RESULTS")
            print("="*60)
            
            # Show warnings first (if any)
            if warnings:
                print(f"\n⚠️  {len(warnings)} WARNING(S) - Classes not in RDFS profile (won't be consumed by typical consumers):")
                print("="*60)
                for w in warnings:
                    print(f"  - {w}")
            
            # Then show errors (if any)
            if errors:
                print(f"\n❌ {len(errors)} ERROR(S) - Validation failures:")
                print("="*60)
                for e in errors:
                    print(f"  - {e}")
                status.value = f"<b>Status:</b> <span style='color: red'>Validation failed ({len(errors)} errors, {len(warnings)} warnings)</span>"
            else:
                print("\n✅ All instance data conforms to RDFS profile!")
                status.value = f"<b>Status:</b> <span style='color: green'>Validation passed{f' ({len(warnings)} warnings)' if warnings else ''}</span>"
                    
        except Exception as e:
            print(f"❌ Error during validation: {e}")
            import traceback
            traceback.print_exc()
            status.value = f"<b>Status:</b> <span style='color: red'>Error</span>"


instance_btn = widgets.Button(description="Select CIM Instance", button_style="info")
add_rdfs_btn = widgets.Button(description="Add RDFS Profile(s)", button_style="info")
clear_rdfs_btn = widgets.Button(description="Clear All Profiles", button_style="warning")
instance_btn.on_click(select_instance_file)
add_rdfs_btn.on_click(add_rdfs_file)
clear_rdfs_btn.on_click(clear_rdfs_files)
validate_btn.on_click(run_validation)

ui = widgets.VBox([
    widgets.HBox([instance_btn, instance_label]),
    widgets.Label("RDFS Profiles:"),
    widgets.HBox([add_rdfs_btn, clear_rdfs_btn]),
    rdfs_list_widget,
    validate_btn,
    status
])

display(ui)
display(output)

Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…